In [3]:
import csv
from gensim import corpora, models, similarities
import numpy as np
from nltk.corpus import stopwords
from six import iteritems
import pyLDAvis.gensim
import re
import string

stops = stopwords.words('english')
keys = ['surveyid', 'additionalinfo']
rows = []
survey_words = []
bad_words = ['oxford', 'consultant', 'the' , 'it', 'id', 'could', 'consultants', '', 'work']
for i in range(10):
    bad_words.append(str(i))
with open('h:/cm_start.csv') as f:
    reader = csv.DictReader(f, fieldnames = keys)
    for row in reader:
        rows.append(row['additionalinfo'])
keys = ['word']
with open('h:/bad_words.csv') as f:
    reader = csv.DictReader(f, fieldnames = keys)
    for row in reader:
        bad_words.append(row['word'].strip())

for row in rows:
    words = row.split()
    good_words = []
    for word in words:
        word = re.sub('[^0-9a-zA-Z]+', '', word).lower()
        if word not in stops and word.lower() not in bad_words:           
            if word != '':
                good_words.append(word)
    if len(good_words) >= 10:
        survey_words.append(good_words)

dictionary = corpora.Dictionary(survey_words)
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1] 
dictionary.filter_tokens(once_ids)
dictionary.compactify()
corpus = [dictionary.doc2bow(resume) for resume in survey_words]
lda = models.LdaModel(corpus = corpus, num_topics = 20, id2word = dictionary, iterations = 2000,passes = 20)
output = lda.print_topics(num_topics = 20)
topic_desc = []
for element in output:
    topic_desc.append({"id" : element[0], "words": element[1]})
topics = [lda.get_document_topics(element, minimum_probability = .05) for element in corpus]
measure = []
for doc in topics:
    for tuple in doc:
        measure.append(tuple[0])

    
data = pyLDAvis.gensim.prepare(topic_model = lda, corpus = corpus, dictionary = dictionary)
pyLDAvis.display(data)

228
